In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## Create the model

In [3]:
class MLP(nn.Module):
    def __init__(self, name, input_size, output_size) -> None:
        super(MLP, self).__init__()
        self.name = name
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, 115)  # linear: y = Ax + b
        self.fc2 = nn.Linear(115, 78)
        self.fc3 = nn.Linear(78, 26)
        self.fc4 = nn.Linear(26, 46)
        self.fc5 = nn.Linear(46, 82)
        self.fc6 = nn.Linear(82, 106)
        self.fc7 = nn.Linear(106, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.relu(x)
        x = self.fc6(x)
        x = F.relu(x)
        x = self.fc7(x)
        
        output = x = F.relu(x)
        return output


Generate fake data (to be replaced with the actual get data functions)

In [4]:
features = np.random.rand(128, 4)  # 4 dimensional features
labels = np.random.rand(128, 5)  # 5 dimensional labels

In [5]:
features = torch.FloatTensor(features)
labels = torch.FloatTensor(labels)

In [6]:
model = MLP('M502', 4, 5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
model.train()
epochs = 20

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()

    # forward pass
    y_pred = model(features)

    # compute loss
    loss = criterion(y_pred, labels)

    print(f'Epoch {epoch}: train loss: {loss.item()}')

    # backward pass
    loss.backward()
    optimizer.step() # updated = old - learning_rate * gradient


100%|██████████| 20/20 [00:00<00:00, 930.74it/s]

Epoch 0: train loss: 0.2362939864397049
Epoch 1: train loss: 0.23488330841064453
Epoch 2: train loss: 0.23493418097496033
Epoch 3: train loss: 0.23593325912952423
Epoch 4: train loss: 0.23677706718444824
Epoch 5: train loss: 0.23673216998577118
Epoch 6: train loss: 0.23582465946674347
Epoch 7: train loss: 0.2345033586025238
Epoch 8: train loss: 0.23320774734020233
Epoch 9: train loss: 0.23223650455474854
Epoch 10: train loss: 0.23170509934425354
Epoch 11: train loss: 0.23157568275928497
Epoch 12: train loss: 0.23173141479492188
Epoch 13: train loss: 0.23203718662261963
Epoch 14: train loss: 0.23237232863903046
Epoch 15: train loss: 0.23265290260314941
Epoch 16: train loss: 0.23282885551452637
Epoch 17: train loss: 0.23288075625896454
Epoch 18: train loss: 0.23281314969062805
Epoch 19: train loss: 0.23264749348163605


## Preparing your data for training with DataLoaders

The `Dataset` retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, and use Python’s `multiprocessing` to speed up data retrieval.

`DataLoader` is an iterable that abstracts this complexity for us in an easy API.

```
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
```

In [13]:
trainset = torch.utils.data.TensorDataset(features, labels)

In [14]:
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [22]:
for epoch in range(epochs):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 4 == 3:  # print every 4 mini batches (?)
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss/4:.3f}\r')
        running_loss = 0.0

print('Finished training')

[1,     1] loss: 0.053
[1,     2] loss: 0.050
[1,     3] loss: 0.055
[1,     4] loss: 0.061
[2,     1] loss: 0.056
[2,     2] loss: 0.058
[2,     3] loss: 0.055
[2,     4] loss: 0.050
[3,     1] loss: 0.051
[3,     2] loss: 0.058
[3,     3] loss: 0.060
[3,     4] loss: 0.050
[4,     1] loss: 0.056
[4,     2] loss: 0.055
[4,     3] loss: 0.050
[4,     4] loss: 0.058
[5,     1] loss: 0.060
[5,     2] loss: 0.055
[5,     3] loss: 0.055
[5,     4] loss: 0.049
[6,     1] loss: 0.054
[6,     2] loss: 0.053
[6,     3] loss: 0.054
[6,     4] loss: 0.058
[7,     1] loss: 0.056
[7,     2] loss: 0.054
[7,     3] loss: 0.054
[7,     4] loss: 0.054
[8,     1] loss: 0.053
[8,     2] loss: 0.059
[8,     3] loss: 0.054
[8,     4] loss: 0.054
[9,     1] loss: 0.052
[9,     2] loss: 0.055
[9,     3] loss: 0.053
[9,     4] loss: 0.058
[10,     1] loss: 0.053
[10,     2] loss: 0.067
[10,     3] loss: 0.054
[10,     4] loss: 0.044
[11,     1] loss: 0.051
[11,     2] loss: 0.054
[11,     3] loss: 0.060
[11,